<a href="https://colab.research.google.com/github/milvus-io/bootcamp/blob/master/bootcamp/tutorials/quickstart/movie_recommendation_with_milvus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>   <a href="https://github.com/milvus-io/bootcamp/blob/master/bootcamp/tutorials/quickstart/movie_recommendation_with_milvus.ipynb" target="_blank">
    <img src="https://img.shields.io/badge/View%20on%20GitHub-555555?style=flat&logo=github&logoColor=white" alt="GitHub Repository"/>

# Movie Recommendation with Milvus
In this notebook, we will explore how to generate embeddings of movie descriptions using OpenAI and leverage those embeddings within Milvus to recommend movies that match your preferences. To enhance our search results, we will utilize filtering to perform metadata searches. The dataset used in this example is sourced from HuggingFace datasets and contains over 8,000 movie entries, providing a rich pool of options for movie recommendations.



## Dependencies and Environment
You can install the dependencies by running the following command:


In [1]:
! pip install openai pymilvus datasets tqdm

> If you are using Google Colab, to enable dependencies just installed, you may need to **restart the runtime** (click on the "Runtime" menu at the top of the screen, and select "Restart session" from the dropdown menu).

We will use OpenAI as the LLM in this example. You should prepare the [api key](https://platform.openai.com/docs/quickstart) `OPENAI_API_KEY` as an environment variable.

In [2]:
import os

os.environ["OPENAI_API_KEY"] = "sk-***********"

## Initialize OpenAI client and Milvus
Initialize the OpenAI client.

In [3]:
from openai import OpenAI

openai_client = OpenAI()

Set the collection name and dimension for the embeddings.

In [4]:
COLLECTION_NAME = "movie_search"
DIMENSION = 1536

BATCH_SIZE = 1000

Connect to Milvus.

In [5]:
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    Collection,
    CollectionSchema,
    DataType,
)

# Connect to Milvus Database
connections.connect(uri="./milvus.db")

> As for the argument of `url` and `token`:
> - Setting the `uri` as a local file, e.g.`./milvus.db`, is the most convenient method, as it automatically utilizes [Milvus Lite](https://milvus.io/docs/milvus_lite.md) to store all data in this file.
> - If you have large scale of data, say more than a million vectors, you can set up a more performant Milvus server on [Docker or Kubernetes](https://milvus.io/docs/quickstart.md). In this setup, please use the server address and port as your uri, e.g.`http://localhost:19530`. If you enable the authentication feature on Milvus, use "<your_username>:<your_password>" as the token, otherwise don't set the token.
> - If you want to use [Zilliz Cloud](https://zilliz.com/cloud), the fully managed cloud service for Milvus, adjust the `uri` and `token`, which correspond to the [Public Endpoint and Api key](https://docs.zilliz.com/docs/on-zilliz-cloud-console#free-cluster-details) in Zilliz Cloud.

In [6]:
# Remove collection if it already exists
if utility.has_collection(COLLECTION_NAME):
    utility.drop_collection(COLLECTION_NAME)

Define the fields for the collection, which include the id, title, type, release year, rating, and description.

In [7]:
# Create collection which includes the id, title, and embedding.
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=64000),
    FieldSchema(name="type", dtype=DataType.VARCHAR, max_length=64000),
    FieldSchema(name="release_year", dtype=DataType.INT64),
    FieldSchema(name="rating", dtype=DataType.VARCHAR, max_length=64000),
    FieldSchema(name="description", dtype=DataType.VARCHAR, max_length=64000),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=DIMENSION),
]
schema = CollectionSchema(fields=fields)
collection = Collection(name=COLLECTION_NAME, schema=schema)

Create the index on the collection and load it.

In [8]:
# Create the index on the collection and load it.
collection.create_index(
    field_name="embedding",
    index_params={"metric_type": "IP", "index_type": "AUTOINDEX", "params": {}},
)
collection.load()

## Dataset
With Milvus up and running we can begin grabbing our data. `Hugging Face Datasets` is a hub that holds many different user datasets, and for this example we are using HuggingLearners's netflix-shows dataset. This dataset contains movies and their metadata pairs for over 8 thousand movies. We are going to embed each description and store it within Milvus along with its title, type, release_year and rating.

In [9]:
import datasets

# Download the dataset
dataset = datasets.load_dataset("hugginglearners/netflix-shows", split="train")

## Insert the Data
Now that we have our data on our machine we can begin embedding it and inserting it into Milvus. The embedding function takes in text and returns the embeddings in a list format. 

In [10]:
def emb_texts(texts):
    res = openai_client.embeddings.create(input=texts, model="text-embedding-3-small")
    return [res_data.embedding for res_data in res.data]

This next step does the actual inserting. We iterate through all the entries and create batches that we insert once we hit our set batch size. After the loop is over we insert the last remaning batch if it exists. 

In [11]:
from tqdm import tqdm

data = [
    [],  # title
    [],  # type
    [],  # release_year
    [],  # rating
    [],  # description
]

# Embed and insert in batches
for i in tqdm(range(0, len(dataset))):
    data[0].append(dataset[i]["title"] or "")
    data[1].append(dataset[i]["type"] or "")
    data[2].append(dataset[i]["release_year"] or -1)
    data[3].append(dataset[i]["rating"] or "")
    data[4].append(dataset[i]["description"] or "")
    if len(data[0]) % BATCH_SIZE == 0:
        data.append(emb_texts(data[4]))
        collection.insert(data)
        data = [[], [], [], [], []]

# Embed and insert the remainder
if len(data[0]) != 0:
    data.append(emb_texts(data[4]))
    collection.insert(data)
    data = [[], [], [], [], []]

100%|██████████| 8807/8807 [01:52<00:00, 78.52it/s] 


## Query the Database
With our data safely inserted into Milvus, we can now perform a query. The query takes in a tuple of the movie description you are searching for and the filter to use. More info about the filter can be found [here](https://milvus.io/docs/boolean.md). The search first prints out your description and filter expression. After that for each result we print the score, title, type, release year, rating and description of the result movies. 

In [12]:
import textwrap


def query(query, top_k=5):
    text, expr = query
    res = collection.search(
        emb_texts(text),
        anns_field="embedding",
        expr=expr,
        param={
            "metric_type": "IP",
            "params": {},
        },
        limit=top_k,
        output_fields=["title", "type", "release_year", "rating", "description"],
    )
    for hit in res:
        print("Description:", text, "Expression:", expr)
        print("Results:")
        for ii, hits in enumerate(hit):
            print(
                "\t" + "Rank:",
                ii + 1,
                "Score:",
                hits.score,
                "Title:",
                hits.entity.get("title"),
            )
            print(
                "\t\t" + "Type:",
                hits.entity.get("type"),
                "Release Year:",
                hits.entity.get("release_year"),
                "Rating:",
                hits.entity.get("rating"),
            )
            print(textwrap.fill(hits.entity.get("description"), 88))
            print()


my_query = ("movie about a fluffly animal", 'release_year < 2019 and rating like "PG%"')

query(my_query)

Description: movie about a fluffly animal Expression: release_year < 2019 and rating like "PG%"
Results:
	Rank: 1 Score: 0.4221611022949219 Title: The Adventures of Tintin
		Type: Movie Release Year: 2011 Rating: PG
This 3-D motion capture adapts Georges Remi's classic comic strip about the adventures
of fearless young journalist Tintin and his trusty dog, Snowy.

	Rank: 2 Score: 0.40418487787246704 Title: Hedgehogs
		Type: Movie Release Year: 2016 Rating: PG
When a hedgehog suffering from memory loss forgets his identity, he ends up on a big
city journey with a pigeon to save his habitat from a human threat.

	Rank: 3 Score: 0.3980240225791931 Title: Osmosis Jones
		Type: Movie Release Year: 2001 Rating: PG
Peter and Bobby Farrelly outdo themselves with this partially animated tale about an
out-of-shape 40-year-old man who's the host to various organisms.

	Rank: 4 Score: 0.3947009742259979 Title: The Lamb
		Type: Movie Release Year: 2017 Rating: PG
A big-dreaming donkey escapes his m